In [1]:
param_len = 54

list_url = ['https://www.atp-autoteile.de/de/search/n-243/p-' + str(_+1) + '/o-1/b-1-592/bremsbelaege-bestprice-atec' for _ in range(param_len)]

print('总数量：' + str(len(list_url)))
print()

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import re

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_part = html.xpath('//a[@class="product-hit-link"]')
                
                # = = = = = = = = = = = = = = =
                
                list_url = ['https://www.atp-autoteile.de' + _.xpath('./@href')[0].strip() for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_tbody = [_.xpath('./descendant::div[@class="product-detail-info"]/table/tbody')[0] for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_kit = []
                for tbody in list_tbody:
                    kit = ''
                    list_tr = tbody.xpath('./tr')
                    for tr in list_tr:
                        if len(tr.xpath('./@class')) != 0:
                            name = re.split(r'^[0-9]+x ', tr.xpath('./td/strong/text()')[0])[1].strip()
                            amount = tr.xpath('./td/strong/text()')[0].replace(name, '').strip()[:-1]
                            kit += '}\n{Part_Name: ' + name + '; Amount: ' + amount + '; '
                        else:
                            if tr.xpath('./td[1]/text()')[0].strip().endswith(':'):
                                kit += tr.xpath('./td[1]/text()')[0].strip() + ' '
                            else:
                                kit += tr.xpath('./td[1]/text()')[0].strip() + ': '
                            if len(tr.xpath('./td[2]/text()')) == 0:
                                kit = kit[:-2] + '; '
                            else:
                                kit += tr.xpath('./td[2]/text()')[0].strip() + '; '

                    if kit.startswith('}'):
                        kit = (kit[2:] + '}').replace('; }', '}')
                    else:
                        kit = (kit + '}').replace('; }', '')

                    list_kit.append(kit)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Menu': url,
                                        'Url': list_url,
                                        'Kit': list_kit})
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Menu': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./menu_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：54

https://www.atp-autoteile.de/de/search/n-243/p-2/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：49
https://www.atp-autoteile.de/de/search/n-243/p-3/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：48
https://www.atp-autoteile.de/de/search/n-243/p-4/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：47
https://www.atp-autoteile.de/de/search/n-243/p-5/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：46
https://www.atp-autoteile.de/de/search/n-243/p-1/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：45
https://www.atp-autoteile.de/de/search/n-243/p-6/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：44
https://www.atp-autoteile.de/de/search/n-243/p-7/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：43
https://www.atp-autoteile.de/de/search/n-243/p-11/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：42
https://www.atp-autoteile.de/de/search/n-243/p-9/o-1/b-1-592/bremsbelaege-bestprice-atec  <->  [ok] - 剩余数量：41
h